# About
* **Author**: Adil Rashitov
* **Created at**: 07.08.2021
* **Goal**: Solve final assignment

![pictures](pictures/FINAL_TASK_1.png)
![Formulation](pictures/OR_2_FINAL.drawio.png)

In [1]:
# Imports / Configs / Global vars

# Import of native python tools
import os
import json
from functools import reduce

# Import of base ML stack libs
import numpy as np
import sklearn as sc

# Multiprocessing for Mac / Linux
import platform
platform.system()
if platform.system() == 'Darwin':
    from multiprocess import Pool
else:
    from multiprocessing import Pool

# Visualization libraries
import plotly.express as px

# Logging configuraiton
import logging
logging.basicConfig(format='[ %(asctime)s ][ %(levelname)s ]: %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Ipython configs
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
display(HTML("<style>.container { width:100% !important; }</style>"))
InteractiveShell.ast_node_interactivity = 'all'

# Pandas configs
import pandas as pd
import geopandas as gpd
pd.options.display.max_rows = 350
pd.options.display.max_columns = 250

# Jupyter configs
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

from ortools.linear_solver import pywraplp

# Data

In [2]:
# Data reading & prepatre
def prepare_data():

    DATA = """
    1	7	None
    2	4	5, 8
    3	6	None
    4	9	None
    5	12	2, 8
    6	8	9
    7	10	10
    8	11	2, 5
    9	8	6
    10	7	7
    11	6	15
    12	8	None
    13	15	None
    14	14	None
    15	3	11
    """


    DATA = DATA.split('\n')[1:-1]
    DATA = list(map(lambda x: x.split('\t'), DATA))
    data = pd.DataFrame(DATA, columns=['Job', 'Processing time', 'Conflicting jobs'])
    return data

df = prepare_data()

In [3]:
N_MACHINES = 3

# Main


## 1. Inputs prepare

In [4]:
jobs_vec = np.array(df['Processing time'].astype(int))


array([ 7,  4,  6,  9, 12,  8, 10, 11,  8,  7,  6,  8, 15, 14,  3])

In [10]:
from ortools.sat.python import cp_model
model = cp_model.CpModel()


# 1. Data definition
jobs_vec = np.array([ 7,  4,  6,  9, 12,  8, 10, 11,  8,  7,  6,  8, 15, 14,  3])
N_JOBS = jobs_vec.shape[0]
N_MACHINES = 3

# 2. Decision matrix definition
X = [ [model.NewBoolVar(f"x_{i}_{j}") for i in range(N_JOBS)] for j in range(N_MACHINES)] 

# 3. 
makespan = model.NewIntVar(0, int(job_vec.sum()), 'makespan')
model.AddMaxEquality(makespan, list(X @ jobs_vec))

model.Minimize(makespan)

TypeError: NotSupported: model.GetOrMakeIndex((((((((((((((((7 * x_0_0) + (4 * x_1_0)) + (6 * x_2_0)) + (9 * x_3_0)) + (12 * x_4_0)) + (8 * x_5_0)) + (10 * x_6_0)) + (11 * x_7_0)) + (8 * x_8_0)) + (7 * x_9_0)) + (6 * x_10_0)) + (8 * x_11_0)) + (15 * x_12_0)) + (14 * x_13_0)) + (3 * x_14_0)))

## 2. Objective funciton

Approaches:
1. Assignment of constrain output `AddMaxEquality` to `obj_value` -> optimization `obj_value`
2. Direct specification of objective function

TypeError: NotSupported: model.GetOrMakeIndex((((((((((((((((7 * x_0_0) + (4 * x_1_0)) + (6 * x_2_0)) + (9 * x_3_0)) + (12 * x_4_0)) + (8 * x_5_0)) + (10 * x_6_0)) + (11 * x_7_0)) + (8 * x_8_0)) + (7 * x_9_0)) + (6 * x_10_0)) + (8 * x_11_0)) + (15 * x_12_0)) + (14 * x_13_0)) + (3 * x_14_0)))

## 3. Constrains
1. **Conflicting jobs constrains**: Jobs can't be processed on the same machine
2. **Unique machine assignment**: Each job assigned at most one worker

### 1. Conflicting jobs constrains

In [23]:
# Jobs data
# 1. Extraction of conflicting jobs
constrains = df.loc[df['Conflicting jobs'] != 'None', ['Job', 'Conflicting jobs']].copy(deep=True)
constrains['Conflicting jobs'] = constrains['Conflicting jobs'].str.split(', ')
constrains = constrains.explode('Conflicting jobs')
constrains = constrains.replace('  ', '', regex=True)

# 2. Extraction of only unique constrain pairs
unique_conflicting_job_pairs = set(list(zip(constrains.T.min(), constrains.T.max())))
constrains = pd.DataFrame(unique_conflicting_job_pairs).astype(int).sort_values([0, 1])



### 2. Unique machine assignment

In [24]:
sum_n_machines_running_job_at_time = list(map(np.sum, np.transpose(d_matrix)))


for n_machines_running_job_at_time in sum_n_machines_running_job_at_time:
    model.Add(n_machines_running_job_at_time <= 1)

In [ ]:
### 